In [2]:
import os
import torch
import numpy as np
import pickle
import json
from imagebind import data
from imagebind.models import imagebind_model
from imagebind.models.imagebind_model import ModalityType
from tqdm import tqdm
# Load the dataset
# with open('/home/sarmistha/Tasks/Missing_help/Others/DATASET/infographicsVQA_train_v1.0.json','r') as f:
#     dataset = json.load(f)



/home/sarmistha/miniconda3/envs/CIKM/lib/python3.9/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/home/sarmistha/miniconda3/envs/CIKM/lib/python3.9/site-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(
/home/sarmistha/miniconda3/envs/CIKM/lib/python3.9/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [2]:
import pandas as pd


dataset=pd.read_csv("full_modality_rag.csv")
dataset.head()

,id,query,answer,image_path,image_summary,retrieved_text_context,retrieved_image_reference
0,1.0,Hi! Mere paas kuch doubts hain credit aur debi...,Of course! Mere ko apni query batao.,NaN,NaN,"Haan, aap card ka upyog karke electricity, pho...",NaN
1,1.0,Whats the best way to use a credit card? Kya m...,Yes! Tumhe hamesha apne credit card ka pura bh...,NaN,NaN,# Summary\nCredit card ek useful financial too...,NaN
2,1.0,"Now batao, roz ke kharche ke liye debit card i...","Well as per my understanding, aap apne debit c...",NaN,NaN,"Haan, aap card ka upyog karke electricity, pho...",NaN
3,1.0,Is it better to use a cashback ya rewards cred...,"Haan, agar aap har mahine balance pura bhar sa...",NaN,NaN,Debit card se immediate access hota hai funds ...,NaN
4,2.0,"Hey, kaise ho? Just kuch doubts the, agar tu u...","Hi, main achha hoon. Shukriya poochhne ke liye...",NaN,NaN,11. Kya usdunited ke nirdharan mein kisi tarah...,NaN


In [8]:
# for row in dataset.itertuples():
#     print(row.Index, row.id, row.query)


In [19]:
# Initialize ImageBind model
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = imagebind_model.imagebind_huge(pretrained=True)
model.eval()
model.to(device)

# Function to process data in batches and extract embeddings
def extract_embeddings(img_paths, queries, ids, batch_size=32):
    embeddings_dict = {
        'ids': [],
        'raw_txt': [],
        'img_path': [],
        'image_embed': [],
        'text_embed': []
    }

    for i in tqdm(range(0, len(img_paths), batch_size)):
        batch_img_paths = img_paths[i:i + batch_size]
        batch_queries = queries[i:i + batch_size]
        batch_ids = ids[i:i + batch_size]

        # Load and transform data
        inputs = {
            ModalityType.VISION: data.load_and_transform_vision_data(batch_img_paths, device),
            ModalityType.TEXT: data.load_and_transform_text(batch_queries, device)
        }

        # Compute embeddings
        with torch.no_grad():
            embeddings = model(inputs)

        # Convert embeddings to numpy arrays
        image_embeddings = embeddings[ModalityType.VISION].cpu().numpy()
        text_embeddings = embeddings[ModalityType.TEXT].cpu().numpy()

        # Append data to the dictionary
        embeddings_dict['ids'].extend(batch_ids)
        embeddings_dict['raw_txt'].extend(batch_queries)
        embeddings_dict['img_path'].extend(batch_img_paths)
        embeddings_dict['image_embed'].extend(image_embeddings)
        embeddings_dict['text_embed'].extend(text_embeddings)

    return embeddings_dict

# Extract embeddings
embeddings_dict = extract_embeddings(valid_img_paths, valid_queries, valid_ids, batch_size=32)

# Save embeddings dictionary as a pickle file
output_file = 'embeddings_database.pkl'
with open(output_file, 'wb') as file:
    pickle.dump(embeddings_dict, file)

print(f"Embeddings have been successfully saved to {output_file}")


100%|██████████| 79/79 [02:45<00:00,  2.09s/it]

Embeddings have been successfully saved to embeddings_database_vaibhav.pkl


In [20]:
with open('embeddings_database.pkl', 'rb') as file:
    embeddings_dict = pickle.load(file)

# Extract image embeddings and corresponding image paths
image_embeddings = np.array(embeddings_dict['image_embed'])
image_paths = embeddings_dict['img_path']

# Normalize image embeddings for efficient cosine similarity
image_norms = np.linalg.norm(image_embeddings, axis=1, keepdims=True)
image_norms = np.clip(image_norms, a_min=1e-10, a_max=None)  # Avoid division by zero
normalized_image_embeddings = image_embeddings / image_norms

# Initialize ImageBind model
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = imagebind_model.imagebind_huge(pretrained=True)
model.eval()
model.to(device)



ImageBindModel(
  (modality_preprocessors): ModuleDict(
    (vision): RGBDTPreprocessor(
      (cls_token): tensor((1, 1, 1280), requires_grad=True)
      
      (rgbt_stem): PatchEmbedGeneric(
        (proj): Sequential(
          (0): PadIm2Video()
          (1): Conv3d(3, 1280, kernel_size=(2, 14, 14), stride=(2, 14, 14), bias=False)
        )
      )
      (pos_embedding_helper): SpatioTemporalPosEmbeddingHelper(
        (pos_embed): tensor((1, 257, 1280), requires_grad=True)
        
      )
    )
    (text): TextPreprocessor(
      (pos_embed): tensor((1, 77, 1024), requires_grad=True)
      (mask): tensor((77, 77), requires_grad=False)
      
      (token_embedding): Embedding(49408, 1024)
    )
    (audio): AudioPreprocessor(
      (cls_token): tensor((1, 1, 768), requires_grad=True)
      
      (rgbt_stem): PatchEmbedGeneric(
        (proj): Conv2d(1, 768, kernel_size=(16, 16), stride=(10, 10), bias=False)
        (norm_layer): LayerNorm((768,), eps=1e-05, elementwise_affine=

In [22]:
# Function to compute the embedding for a query text
def get_query_embedding(query_text):
    inputs = {
        ModalityType.TEXT: data.load_and_transform_text([query_text], device)
    }
    with torch.no_grad():
        embeddings = model(inputs)
    return embeddings[ModalityType.TEXT].cpu().numpy()

# Function to retrieve top-k similar images
def retrieve_topk_similar_images(query_embedding, k=5):
    query_embedding = query_embedding.flatten()
    query_norm = np.linalg.norm(query_embedding)
    if query_norm == 0:
        normalized_query = query_embedding
    else:
        normalized_query = query_embedding / query_norm
    similarities = np.dot(normalized_image_embeddings, normalized_query)
    topk_indices = np.argsort(similarities)[-k:][::-1]  # Descending order
    topk_scores = similarities[topk_indices]
    return topk_indices, topk_scores


import pandas as pd


dataset=pd.read_csv("full_modality_rag.csv")
base_image_dir = '/Path'
# dataset.head()
# Define top_k values
#top_k_values = [5, 10, 15, 20]
top_k_values = [1,2,3,4]



# Process each split and top_k
# for split, split_path in splits.items():
#     print(f"\nProcessing {split} split...")
#     with open(split_path, 'r') as f:
#         dataset = json.load(f)['data']

for top_k in top_k_values:
    print(f"  Retrieving top-{top_k}...")
    retrieval_results = {}

    # for sample in tqdm(dataset):
    for row in dataset.itertuples():
        if pd.isna(row.query):
            continue
        query_text = row.query
        query_id = row.id
        query_embedding = get_query_embedding(query_text)
        topk_indices, topk_scores = retrieve_topk_similar_images(query_embedding, k=top_k)
        retrieved_image_paths = [image_paths[i] for i in topk_indices]
        retrieval_results[query_id] = {
            'question': query_text,
            'retrieved_image_paths': retrieved_image_paths,
            'similarity_scores': topk_scores.tolist()
            }

        # Save results
    output_file = f'retrieval_infovqa_top{top_k}_vaibhav.pkl'
    with open(output_file, 'wb') as file:
        pickle.dump(retrieval_results, file)
    print(f"  Saved to {output_file}")

  Retrieving top-1...
  Saved to retrieval_infovqa_top1_vaibhav.pkl
  Retrieving top-2...
  Saved to retrieval_infovqa_top2_vaibhav.pkl
  Retrieving top-3...
  Saved to retrieval_infovqa_top3_vaibhav.pkl
  Retrieving top-4...
  Saved to retrieval_infovqa_top4_vaibhav.pkl


In [3]:
with open('retrieval.pkl', "rb") as f:

    db = pickle.load(f)

In [4]:
print(db[])

{1.0: {'question': 'Is it better to use a cashback ya rewards credit card for all my purchases instead of cash ya debit?', 'retrieved_image_paths': ['/home/sarmistha/Sarmistha/ECML/extracted_images/banking/application-for-loan-overdraft-against-securities-05jan-22_page-0012.jpg', '/home/sarmistha/Sarmistha/ECML/extracted_images/banking/application-for-loan-overdraft-against-securities-05jan-22_page-0012.jpg', '/home/sarmistha/Sarmistha/ECML/extracted_images/banking/application-for-loan-overdraft-against-securities-05jan-22_page-0012.jpg', '/home/sarmistha/Sarmistha/ECML/extracted_images/banking/application-for-loan-overdraft-against-securities-05jan-22_page-0012.jpg'], 'similarity_scores': [0.21782541275024414, 0.21782541275024414, 0.21782541275024414, 0.21782541275024414]}, 2.0: {'question': 'So, kya main sirf apne debit card ka istemal karke fees ke liye stick karun?', 'retrieved_image_paths': ['/home/sarmistha/Sarmistha/ECML/extracted_images/banking/withdrawal slip.png', '/home/sarm

In [ ]:
CUDA_VISIBLE_DEVICES=6,7 accelerate launch --num_processes=2 your_script.py
CUDA_VISIBLE_DEVICES=6,7 python -c "import torch; print(torch.cuda.device_count()); print(torch.cuda.get_device_name(0)); print(torch.cuda.get_device_name(1))"